### 抽象--使用高阶函数
- 过程（函数）作为参数
- 过程（函数）作为返回值

In [1]:
# 课程之中的示例, 由于next是python关键字，所以使用then

def sum(term, a, then, b):
    if a > b:
        return 0
    else:
        return term(a) + sum(term, then(a), then, b)


## 数值积分的一个近似函数
def integral(f, a, b, dx):
    def add_dx(x):
        return x + dx
    
    return sum(f, a + dx / 2.0, add_dx, b) * dx

def cube(x):
    return x * x * x

print(integral(cube, 0, 1, 0.002))
print(integral(cube, 0, 1, 0.0006))


0.24999950000000098
0.2502000149900124


In [5]:
# 辛普森数值积分方法
def is_even(n):
    return n % 2 == 0


def sum(f, a, param, n, k):
    if k == n:
        return f(param(k))
    elif k == 0:
        return f(param(k)) + sum(f, a, param, n, 1)
    else:
        if is_even(k):
            return 2 * f(param(k)) + sum(f, a, param, n, k + 1)
        else:
            return 4 * f(param(k)) + sum(f, a, param, n, k + 1)


def simpson_integral(f, a, b, n):
    def param(k):
        return a + k * (b - a) / n
    
    return sum(f, a, param, n, 0) * (b - a) / (3 * n)

print(simpson_integral(cube, 0, 1, 300))
print(simpson_integral(cube, 0, 1, 2000))

## 从结果来说。辛普森方法确实更加准确


0.25000000000000017
0.2500000000000003


In [13]:
## python 尾递归的测试

def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n - 1)  # 具有表达式，因此需要维护当前的函数栈
    

def factorial_ite(n, k, product):
    if k == n:
        return product * k
    else:
       return factorial_ite(n, k + 1, product * k) 

def factorial_i(a):
    return factorial_ite(a, 1, 1)

print(factorial_i(200))

# python 本身不支持尾递归的优化，因此就算函数具有尾递归的形式，其依旧不会释放函数栈
# print(factorial_i(100000))  出错


788657867364790503552363213932185062295135977687173263294742533244359449963403342920304284011984623904177212138919638830257642790242637105061926624952829931113462857270763317237396988943922445621451664240254033291864131227428294853277524242407573903240321257405579568660226031904170324062351700858796178922222789623703897374720000000000000000000000000000000000000000000000000


In [ ]:
## custom tail recurse
import sys
sys.set_int_max_str_digits(10000000)


def tail_opt(g):
    class TailRecurseException(BaseException):
        def __init__(self, args, kwargs):
            self.args = args
            self.kwargs = kwargs

    def func(*args, **kwargs):
        f = sys._getframe()
        if f.f_back and f.f_back.f_back and f.f_back.f_back.f_code == f.f_code:
            raise TailRecurseException(args, kwargs)
        else:
            while 1:
                try:
                    return g(*args, **kwargs)
                except TailRecurseException as e:
                    # 捕获异常, 拿到参数, 退出被修饰函数的递归调用栈
                    args = e.args
                    kwargs = e.kwargs

    func.__doc__ = g.__doc__
    return func


@tail_opt
def factorial_i1(n, acc=1):
    "calculate a factorial"
    if n == 0:
        return acc
    return factorial_i1(n-1, n*acc)
# print(factorial_i1(20010))

: 